# 管理`Prompt`大小

通过 Agent 调用工具，或者利用 RAG 和历史对话等方式增强上下文，可能导致提示词变得比模型上下文窗口更大。因此，需要管理好提示词的大小，以避免模型内存溢出。

In [1]:
import os
from operator import itemgetter

from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

os.environ["http_proxy"] = "http://127.0.0.1:10801"
os.environ["https_proxy"] = "http://127.0.0.1:10801"


In [2]:

wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=10_000)
)
tools = [wiki]

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo")

## 尝试一个不处理提示词大小的例子

In [13]:
agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind_functions(tools)
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'List of current United States governors'}`


Page: List of current United States governors
Summary: The United States has 50 states and 5 territories that each elect a governor to serve as chief executive of the state or territorial government. The sole federal district, the District of Columbia, elects a mayor to oversee their government in a similar manner. Most states and territories also have a lieutenant governor who is the second-highest-ranking official and the first to succeed the governor in the event of a vacancy.As of April 4, 2024, there are 27 states with Republican governors and 23 states with Democratic governors. Four U.S. territories have Democratic governors, while one has an independent governor. Pedro Pierluisi of Puerto Rico is a member of the New Progressive Party, although he is also affiliated with the Democratic Party. All 55 governors are members of the National Governors Associati

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4633 tokens (4565 in the messages, 68 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

## 添加提示词处理函数

删除历史记录中最早的消息对。

In [4]:
def condense_prompt(prompt: ChatPromptValue) -> ChatPromptValue:
    print(prompt)
    messages = prompt.to_messages()
    num_tokens = llm.get_num_tokens_from_messages(messages)
    ai_function_messages = messages[2:]
    while num_tokens > 4_000:
        ai_function_messages = ai_function_messages[2:]
        num_tokens = llm.get_num_tokens_from_messages(
            messages[:2] + ai_function_messages
        )
    messages = messages[:2] + ai_function_messages
    return ChatPromptValue(messages=messages)


agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | condense_prompt
    | llm.bind_functions(tools)
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [18]:
agent.get_graph().print_ascii()

              +---------------------------------------+      
              | Parallel<input,agent_scratchpad>Input |      
              +---------------------------------------+      
                        ***               ***                
                     ***                     ***             
                   **                           **           
+-----------------------------+              +-------------+ 
| Lambda(itemgetter('input')) |              | Lambda(...) | 
+-----------------------------+              +-------------+ 
                        ***               ***                
                           ***         ***                   
                              **     **                      
             +----------------------------------------+      
             | Parallel<input,agent_scratchpad>Output |      
             +----------------------------------------+      
                                  *                          
        

In [19]:
agent.get_prompts()

[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])]

In [5]:
agent_executor.invoke(
    {
        "input": "Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?"
    }
)



> Entering new AgentExecutor chain...
messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content="Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?")]

Invoking: `wikipedia` with `{'query': 'Current President of the United States'}`


Page: President of the United States
Summary: The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansion

{'input': "Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?",
 'output': 'The current US president is Joe Biden. His home state is Delaware. The official bird of Delaware is the Delaware Blue Hen. The scientific name of the Delaware Blue Hen is Gallus gallus.'}